# Exploring the Roman Senators in Wikidata

This Jupyter notebook explores Wikidata to find the number of roman senators, what information is documented on them, and see if this population can be used for some analysis.

## SPARQL wrapper

This is the python script to get SPARQL queries in a specific SPARQL Endpoint.

In [4]:
from typing import List, Dict
from SPARQLWrapper import SPARQLWrapper, JSON, SPARQLExceptions
from urllib.error import HTTPError
import pandas as pd

def __handle_row(row: Dict[str, dict]) -> Dict[str, str]:
    """Transform an object coming from a SPARQL query (through SPARQLWrapper) into a dictionnary for better use."""

    obj: Dict[str, str] = {}
    for key in row.keys():
        obj[key] = row[key]["value"]
    return obj

def query(sparql_url: str, request: str) -> List[Dict[str, str]]:
    """
    Execute the given request on the given endpoint
    Request needs to be only SELECT: won't work if it is a INSERT or DELETE request.
    """

    # Init the endpoint
    sparql_endpoint = SPARQLWrapper(
        sparql_url,
        agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    )
    sparql_endpoint.setReturnFormat(JSON)

    # Prepare the query
    sparql_endpoint.setQuery(request)

    # DEBUG
    # print('==============')
    # print(request)

    # Execute the query, handles errors,
    try: 
        response = sparql_endpoint.queryAndConvert()["results"]["bindings"]
    except SPARQLExceptions.QueryBadFormed as error:
        print(error.msg)
        return False
    except HTTPError as error:
        raise Exception(f"HTTP Error {error.code}: {error.reason}")
    # and transform the object
    response = list(map(__handle_row, response))

    # If the answer is empty, return an actual empty array
    if response == [{}]: 
        return []
    
    return response


def execute(sparql_url: str, request: str) -> None:
    """
    Execute the given request against the previously set endpoint.
    Request needs to be only INSERTs or DELETEs.
    """
    
    # Init the endpoint
    sparql_endpoint = SPARQLWrapper(sparql_url)
        
    # Prepare the query
    sparql_endpoint.setQuery(request)
    sparql_endpoint.method = "POST"

    # DEBUG
    # print('==============')
    # print(request)

    # Execute the query
    try: 
        sparql_endpoint.query()
    except SPARQLExceptions.QueryBadFormed as error:
        print(error.msg)
        return False
    except HTTPError as error:
        raise Exception(f"HTTP Error {error.code}: {error.reason}")



def run(sparql_url: str, query_string: str) -> List[Dict[str, str]]:
    """ Wrapper of "query" and "execute" function."""

    if 'delete' in query_string.lower() or 'insert' in query_string.lower():
        return execute(sparql_url, query_string)
    elif 'select' in query_string.lower():
        return pd.DataFrame(data=query(sparql_url, query_string))
    else:
        raise Exception('Query error: Only "SELECT", "INSERT", "DELETE" are supported.')


## Get the number of Roman Senators

This query gets the number of senators.

Senators are individuals that have as a function "Roman Senator" (Q20056508).

In [5]:
run("https://query.wikidata.org/sparql",
    """ 
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>

SELECT (count(*) as ?eff) # What is the difference between ?number and ?eff?
WHERE {
    ?person wdt:P39 wd:Q20056508.
}
"""
)

,eff
0,2434


We find 2434 roman senators throughout the whole Wikidata KG.

## Exploring the attributes of those roman seantors

We can now explore and see what are the properties of those roman seantors, 

In [6]:
run("https://query.wikidata.org/sparql",
    """ 
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>

SELECT ?p ?propLabel ?eff
WHERE {
{
SELECT ?p  (count(*) as ?eff)
WHERE {
    ?item wdt:P39 wd:Q20056508.
  
    ?item wdt:P31 wd:Q5. # Any instance of a human. The query seems faster with specifying it
            
    ?item ?p ?o.
    }
GROUP BY ?p 

    }
?prop wikibase:directClaim ?p .

SERVICE wikibase:label { bd:serviceParam wikibase:language "en". } 


}  
ORDER BY DESC(?eff)
"""
)

,p,propLabel,eff
0,http://www.wikidata.org/prop/direct/P39,position held,6219
1,http://www.wikidata.org/prop/direct/P106,occupation,4142
2,http://www.wikidata.org/prop/direct/P1343,described by source,3329
3,http://www.wikidata.org/prop/direct/P2348,time period,2749
4,http://www.wikidata.org/prop/direct/P2359,nomen gentilicium,2447
...,...,...,...
608,http://www.wikidata.org/prop/direct/P11158,Dimitri and Aliki Perrotis Central Library ID,1
609,http://www.wikidata.org/prop/direct/P11179,MSVF ID,1
610,http://www.wikidata.org/prop/direct/P11493,Biblissima authority ID,1
611,http://www.wikidata.org/prop/direct/P11936,Biblioteca centrale Cappuccini authority ID,1


We can see that the position held is the most documented information on those roman senators. This could be investigated further to see if other positions in the cursus honorum (who has been consul, etc.)

What seems also interesting is the Gens documented. This could be intresting to investigate. Which Gens produced the most senators.

Other information of interest:
- Family: who is the father, has the father already been a senator (who is Homo Novus)

Roman Consul: Q40779